# Merge TME celularity estimations with Leukocyte scores

Script related to figure s3c

### Aim:
- Merge TME cellularity scores using leukocyte methylation data for TCGAov data

In [1]:
!python --version

Python 3.6.8 :: Anaconda custom (64-bit)


## Import packages

In [1]:
import pandas as pd

In [2]:
home = "~/git_repos/HGSOC_TME_Heterogeneity/"

## Load leukocyte methylation data

In [3]:
luk_data = home + 'Data/3/TCGA_all_leuk_estimate.masked.20170107.tsv'

luk_data = pd.read_csv(luk_data,
                       sep='\t',
                       header=None)

luk_data.head()

,0,1,2
0,ACC,TCGA-OR-A5J1-01A-11D-A29J-05,0.046374
1,ACC,TCGA-OR-A5J2-01A-11D-A29J-05,0.057859
2,ACC,TCGA-OR-A5J3-01A-11D-A29J-05,0.048460
3,ACC,TCGA-OR-A5J4-01A-11D-A29J-05,0.043988
4,ACC,TCGA-OR-A5J5-01A-11D-A29J-05,0.016759


In [4]:
luk_data[2].min()

-9.29722165644888e-05

### NOTE: We need positive values for the box cox model
### Remove samples with negative values, since score should go from 0 - 1

In [5]:
(luk_data[2] <= 0).value_counts()

False    10816
True         1
Name: 2, dtype: int64

In [6]:
luk_data = luk_data[luk_data[2] > 0]

In [7]:
(luk_data[2] <= 0).value_counts()

False    10816
Name: 2, dtype: int64

In [8]:
luk_data[2].min()

0.0012519711455424

In [9]:
luk_data['sampleID'] = luk_data[1].str.replace('-','.').str.slice(0,15)

In [10]:
luk_data.set_index('sampleID',
                   inplace=True)

In [11]:
luk_data.head()

,0,1,2
sampleID,,,
TCGA.OR.A5J1.01,ACC,TCGA-OR-A5J1-01A-11D-A29J-05,0.046374
TCGA.OR.A5J2.01,ACC,TCGA-OR-A5J2-01A-11D-A29J-05,0.057859
TCGA.OR.A5J3.01,ACC,TCGA-OR-A5J3-01A-11D-A29J-05,0.048460
TCGA.OR.A5J4.01,ACC,TCGA-OR-A5J4-01A-11D-A29J-05,0.043988
TCGA.OR.A5J5.01,ACC,TCGA-OR-A5J5-01A-11D-A29J-05,0.016759


In [12]:
luk_ovdata = luk_data[luk_data[0] == 'OV']

luk_ovdata.head()

,0,1,2
sampleID,,,
TCGA.04.1331.01,OV,TCGA-04-1331-01A-01D-0432-05,0.071377
TCGA.04.1332.01,OV,TCGA-04-1332-01A-01D-0432-05,0.122724
TCGA.04.1335.01,OV,TCGA-04-1335-01A-01D-0432-05,0.101598
TCGA.04.1336.01,OV,TCGA-04-1336-01A-01D-0432-05,0.083110
TCGA.04.1337.01,OV,TCGA-04-1337-01A-01D-0432-05,0.126009


In [13]:
luk_ovdata.drop(labels=[0,1],
                axis='columns',
                inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
luk_ovdata.head()

,2
sampleID,
TCGA.04.1331.01,0.071377
TCGA.04.1332.01,0.122724
TCGA.04.1335.01,0.101598
TCGA.04.1336.01,0.083110
TCGA.04.1337.01,0.126009


In [15]:
luk_ovdata.rename(columns={2:'LeukocyteMeth_score'},
                  inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [16]:
luk_ovdata.head()

,LeukocyteMeth_score
sampleID,
TCGA.04.1331.01,0.071377
TCGA.04.1332.01,0.122724
TCGA.04.1335.01,0.101598
TCGA.04.1336.01,0.083110
TCGA.04.1337.01,0.126009


In [17]:
luk_ovdata['LeukocyteMeth_score'].min()

0.0026953869762475604

### Drop duplicate samples in index

In [18]:
luk_ovdata = luk_ovdata[luk_ovdata.index.isin(luk_ovdata.index.drop_duplicates(keep=False))]

luk_ovdata.head()

,LeukocyteMeth_score
sampleID,
TCGA.04.1331.01,0.071377
TCGA.04.1332.01,0.122724
TCGA.04.1335.01,0.101598
TCGA.04.1336.01,0.083110
TCGA.04.1337.01,0.126009


In [19]:
len(luk_ovdata)

610

## Load TME cell estimations

### Bindea

In [20]:
bin_data = home + 'Data/3/BindeaImmuneCells_TCGAov_NES.txt'

bin_data = pd.read_csv(bin_data,
                       sep='\t',
                       index_col='Term').T

bin_data.drop(labels=['Angiogenesis','Antigen_presentation_machinery'],
              axis='columns',
              inplace=True)

bin_data.head()

Term,activated_Dendritic_cells,B_cells,CD8_T_cells,Citotoxic_cells,Dendritic_cells,Eosinophils,immature_Dendritic_cells,Macrophages,Mast_cells,Neutrophils,...,T_cells,T_helper_cells,T_central_memory_cells,T_effector_memory_cells,T_follicular_helper_cells,T_gamma_delta_cells,Th1_cells,Th17_cells,Th2_cells,Treg_cells
TCGA.04.1348.01,0.120802,-0.039622,0.227233,0.031497,-0.015982,0.080828,0.070310,0.157980,-0.137755,-0.049221,...,-0.000596,0.254706,0.152481,0.108996,0.005709,-0.236398,0.029935,-0.041889,0.067944,-0.028578
TCGA.04.1357.01,0.105562,0.029036,0.263201,0.078787,0.039193,0.108250,0.106248,0.202552,-0.138042,-0.019741,...,0.162230,0.256873,0.167683,0.140040,0.054486,-0.175154,0.042390,0.014575,0.006402,-0.033000
TCGA.04.1362.01,0.133338,-0.098435,0.208848,-0.074348,-0.028652,0.116464,0.055821,0.138286,-0.129137,-0.091866,...,-0.139158,0.245309,0.174574,0.116130,-0.004587,-0.230443,0.001276,-0.128269,0.024894,-0.103476
TCGA.04.1364.01,-0.053156,-0.071953,0.193288,-0.163497,-0.143444,0.088167,-0.011111,0.036981,-0.111635,-0.153664,...,-0.201852,0.262310,0.105963,0.115297,0.000454,-0.212713,-0.000375,-0.136518,0.069309,-0.226078
TCGA.04.1365.01,0.190244,-0.091763,0.209855,-0.005047,-0.034158,0.085352,0.076296,0.175680,-0.150940,-0.048484,...,-0.016294,0.273239,0.135881,0.119548,0.028944,-0.246262,0.033078,-0.102564,0.083255,-0.081974


### Davoli

In [21]:
dav_data = home + 'Data/3/DavoliImmuneCells_TCGAov_NES.txt'

dav_data = pd.read_csv(dav_data,
                       sep='\t',
                       index_col='Term').T

dav_data.head()

Term,CD4_mature,CD8_effector,NK_cells,B_cells,T_regs,Dendritics,CD8_effector_NK_cells,Macrophages,Macrophages_M2,Macrophages_M1
TCGA.04.1348.01,0.022657,-0.133686,-0.139781,-0.045055,-0.014131,0.132643,-0.010075,0.413101,0.262198,0.369973
TCGA.04.1357.01,0.103612,0.051358,0.004287,0.148263,-0.014680,0.148268,0.129134,0.450268,0.304542,0.433461
TCGA.04.1362.01,-0.124390,-0.369165,-0.346159,-0.209140,-0.095841,-0.049672,-0.217201,0.333928,0.169423,0.170850
TCGA.04.1364.01,-0.182340,-0.436440,-0.467786,-0.127647,-0.222235,-0.097677,-0.358318,-0.022455,-0.080364,-0.112248
TCGA.04.1365.01,-0.042293,-0.155227,-0.146163,-0.005805,-0.108218,0.122148,-0.045398,0.390748,0.213862,0.348351


### Danaher

In [22]:
dan_data = home + 'Data/3/DanaherImmuneCells_TCGAov_NES.txt'

dan_data = pd.read_csv(dan_data,
                       sep='\t',
                       index_col='Term').T

dan_data.head()

Term,B.cells,CD45,CD8.T.cells,Cytotoxic.cells,DC,Exhausted.CD8,Macrophages,Mast.cells,Neutrophils,NK.CD56dim.cells,NK.cells,T.cells,Th1.cells,Treg
TCGA.04.1348.01,-0.213772,0.237833,0.038574,-0.004140,-0.136734,-0.062459,0.210535,-0.307145,-0.188507,-0.236342,-0.204308,-0.043640,-0.183136,-0.027349
TCGA.04.1357.01,-0.089404,0.422715,0.236438,0.072817,-0.147556,0.032395,0.231140,-0.317968,-0.160218,-0.219044,-0.166882,0.097841,-0.108760,-0.031581
TCGA.04.1362.01,-0.343642,0.006612,-0.226110,-0.209050,-0.232155,-0.209200,0.050957,-0.335683,-0.212186,-0.363609,-0.312521,-0.232933,-0.319826,-0.099027
TCGA.04.1364.01,-0.189344,-0.248201,-0.239439,-0.334968,-0.263151,-0.310654,-0.187723,-0.386647,-0.322938,-0.450434,-0.291942,-0.313260,-0.528989,-0.216356
TCGA.04.1365.01,-0.231293,0.175730,-0.021701,-0.037648,-0.180634,-0.037813,0.163545,-0.339051,-0.145892,-0.258236,-0.259060,-0.049737,-0.199905,-0.078449


### ConsensusTME

In [23]:
con_data = home + 'Data/3/ConsensusTME_TCGAov_NES.txt'

con_data = pd.read_csv(con_data,
                       sep='\t',
                       index_col='Term').T

con_data.head()

Term,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
TCGA.04.1348.01,0.116094,0.015198,0.177872,-0.021979,0.163273,-0.026367,0.308360,0.265947,0.263728,0.227799,0.156372,0.040087,-0.042706,0.328188,0.081508,0.097530,0.064612,0.111321
TCGA.04.1357.01,0.222905,0.142556,0.264784,0.018350,0.254335,0.120839,0.335431,0.325181,0.353313,0.315416,0.237937,0.200891,0.022803,0.392530,0.235109,0.247585,0.215137,0.264070
TCGA.04.1362.01,-0.053375,-0.313831,-0.012503,-0.103150,-0.012369,-0.080783,0.140036,0.075724,0.076199,-0.024923,0.037650,-0.198247,-0.157238,0.236237,-0.127952,-0.126697,-0.173251,-0.171235
TCGA.04.1364.01,-0.242056,-0.479257,-0.241447,-0.133003,-0.177319,-0.101909,-0.041094,-0.120676,-0.091007,-0.125996,-0.196706,-0.399216,-0.361281,0.117505,-0.298982,-0.283349,-0.286931,-0.398547
TCGA.04.1365.01,0.043992,-0.059181,0.126954,-0.035426,0.137609,-0.023786,0.265740,0.244971,0.217097,0.192818,0.127383,-0.014752,-0.029825,0.333373,0.027088,0.039795,0.031051,0.039815


### CIBERSORT

In [24]:
cib_data = home + 'Data/3/CIBERSORT_absolute_TCGAov_Results.txt'

cib_data = pd.read_csv(cib_data,
                       sep='\t',
                       index_col='Mixture')

cib_data.index = cib_data.index.str.replace('-','.')

cib_data.drop(labels=['P-value','Correlation','RMSE','Absolute score (sig.score)'],
              axis='columns',
              inplace=True)

cib_data.head()

,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells follicular helper,T cells regulatory (Tregs),T cells gamma delta,...,Monocytes,Macrophages M0,Macrophages M1,Macrophages M2,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils
Mixture,,,,,,,,,,,,,,,,,,,,,
TCGA.04.1348.01,0.000795,0.000830,0.000000,0.025868,0.0,0.038003,0.0,0.043519,0.026832,0.0,...,0.005156,0.104983,0.024237,0.062198,0.000000,0.000000,0.000000,0.014234,0.0,0.000000
TCGA.04.1357.01,0.015474,0.000000,0.000000,0.083101,0.0,0.120940,0.0,0.053592,0.052900,0.0,...,0.006448,0.057623,0.062343,0.124707,0.001035,0.000000,0.014600,0.000000,0.0,0.000000
TCGA.04.1362.01,0.008684,0.000000,0.000058,0.000000,0.0,0.049975,0.0,0.010518,0.000000,0.0,...,0.016486,0.000000,0.004502,0.058745,0.000000,0.017518,0.000000,0.001998,0.0,0.000000
TCGA.04.1364.01,0.018040,0.000000,0.001023,0.000000,0.0,0.004297,0.0,0.001651,0.002840,0.0,...,0.000000,0.017076,0.000000,0.005596,0.000000,0.000000,0.000046,0.000000,0.0,0.000000
TCGA.04.1365.01,0.001747,0.000229,0.000000,0.000000,0.0,0.072325,0.0,0.048944,0.020026,0.0,...,0.003811,0.080549,0.054699,0.034943,0.000000,0.000000,0.002676,0.000000,0.0,0.000605


### MCPcounter

In [25]:
mcp_data = home + 'Data/3/MCPcounterImmuneCells_TCGAov_Results.txt'

mcp_data = pd.read_csv(mcp_data,
                       sep='\t',
                       index_col='Term').T

mcp_data.head()

Term,T cells,CD8 T cells,Cytotoxic lymphocytes,NK cells,B lineage,Monocytic lineage,Myeloid dendritic cells,Neutrophils,Endothelial cells,Fibroblasts
TCGA.04.1348.01,140.379514,82.0113,122.446500,5.310562,61.407637,552.419614,34.31030,185.031820,227.841865,5541.236563
TCGA.04.1357.01,284.691221,326.8229,289.747943,10.823550,220.784525,897.167057,154.29686,245.269087,204.511719,9558.854162
TCGA.04.1362.01,217.392593,5.2953,20.046643,5.088475,38.763562,313.796143,12.97356,193.257807,257.996087,3400.404175
TCGA.04.1364.01,41.826871,19.9291,1.877800,4.187238,98.479450,114.849786,5.85152,158.810920,236.205152,2706.700838
TCGA.04.1365.01,167.179664,66.2950,107.277386,7.533512,92.021987,462.092157,40.43996,185.947460,201.272332,3607.766325


### TIMER

In [26]:
tim_data = home + 'Data/3/TIMERImmuneCells_TCGAov_Results.csv'

tim_data = pd.read_csv(tim_data,
                       sep=',',
                       index_col='sampleID')

tim_data.index = tim_data.index.str.replace('-','.')

tim_data.head()

,B_cell,T_cell.CD4,T_cell.CD8,Neutrophil,Macrophage,DC
sampleID,,,,,,
TCGA.04.1348.01,0.284869,0.206343,0.138018,0.187014,0.000000,0.575930
TCGA.04.1357.01,0.605259,0.259462,0.024222,0.100005,0.000000,0.806742
TCGA.04.1362.01,0.159977,0.144421,0.263358,0.100429,0.107244,0.338729
TCGA.04.1364.01,0.149258,0.215276,0.022342,0.000000,0.041678,0.169301
TCGA.04.1365.01,0.041258,0.086683,0.282746,0.195290,0.000000,0.533185


### xCELL

In [27]:
xce_data = home + 'Data/3/xCell_TCGAov_RNAseq_Results.txt'

xce_data = pd.read_csv(xce_data,
                       sep='\t',
                       index_col='Term').T

xce_data.drop(labels=['ImmuneScore','StromaScore','MicroenvironmentScore'],
              axis='columns',
              inplace=True)

xce_data.head()

Term,Adipocytes,Astrocytes,B-cells,Basophils,CD4+ T-cells,CD4+ Tcm,CD4+ Tem,CD4+ memory T-cells,CD4+ naive T-cells,CD8+ T-cells,...,Th2 cells,Tregs,aDC,cDC,iDC,ly Endothelial cells,mv Endothelial cells,naive B-cells,pDC,pro B-cells
TCGA.04.1348.01,0.0000,0.0494,0.0535,0.1007,0.0,0.0000,0.0000,0.0074,0.0000,0.0264,...,0.1211,0.0000,0.3265,0.0287,0.0,0.0000,0.0098,0.0,0.0199,0.0442
TCGA.04.1357.01,0.0000,0.0301,0.1330,0.0486,0.0,0.0034,0.0458,0.0087,0.0227,0.1359,...,0.0081,0.0057,0.3852,0.1140,0.0,0.0000,0.0037,0.0,0.0382,0.0015
TCGA.04.1362.01,0.0000,0.0506,0.0039,0.0393,0.0,0.0239,0.0000,0.0000,0.0000,0.0000,...,0.0780,0.0000,0.1563,0.0125,0.0,0.0019,0.0032,0.0,0.0070,0.0116
TCGA.04.1364.01,0.0016,0.0654,0.0000,0.0452,0.0,0.0038,0.0000,0.0000,0.0000,0.0000,...,0.1076,0.0000,0.0000,0.0000,0.0,0.0021,0.0117,0.0,0.0000,0.0390
TCGA.04.1365.01,0.0000,0.0612,0.0179,0.0906,0.0,0.0000,0.0000,0.0121,0.0000,0.0000,...,0.1709,0.0000,0.2709,0.0289,0.0,0.0000,0.0039,0.0,0.0155,0.0276


## Merge leukocyte score with TME cell estimations

In [28]:
bin_data = pd.concat([luk_ovdata,bin_data], axis='columns', join='inner')

bin_data.head()

,LeukocyteMeth_score,activated_Dendritic_cells,B_cells,CD8_T_cells,Citotoxic_cells,Dendritic_cells,Eosinophils,immature_Dendritic_cells,Macrophages,Mast_cells,...,T_cells,T_helper_cells,T_central_memory_cells,T_effector_memory_cells,T_follicular_helper_cells,T_gamma_delta_cells,Th1_cells,Th17_cells,Th2_cells,Treg_cells
TCGA.04.1348.01,0.150949,0.120802,-0.039622,0.227233,0.031497,-0.015982,0.080828,0.070310,0.157980,-0.137755,...,-0.000596,0.254706,0.152481,0.108996,0.005709,-0.236398,0.029935,-0.041889,0.067944,-0.028578
TCGA.04.1357.01,0.267944,0.105562,0.029036,0.263201,0.078787,0.039193,0.108250,0.106248,0.202552,-0.138042,...,0.162230,0.256873,0.167683,0.140040,0.054486,-0.175154,0.042390,0.014575,0.006402,-0.033000
TCGA.04.1362.01,0.084071,0.133338,-0.098435,0.208848,-0.074348,-0.028652,0.116464,0.055821,0.138286,-0.129137,...,-0.139158,0.245309,0.174574,0.116130,-0.004587,-0.230443,0.001276,-0.128269,0.024894,-0.103476
TCGA.04.1364.01,0.033465,-0.053156,-0.071953,0.193288,-0.163497,-0.143444,0.088167,-0.011111,0.036981,-0.111635,...,-0.201852,0.262310,0.105963,0.115297,0.000454,-0.212713,-0.000375,-0.136518,0.069309,-0.226078
TCGA.04.1365.01,0.141471,0.190244,-0.091763,0.209855,-0.005047,-0.034158,0.085352,0.076296,0.175680,-0.150940,...,-0.016294,0.273239,0.135881,0.119548,0.028944,-0.246262,0.033078,-0.102564,0.083255,-0.081974


In [29]:
bin_data.to_csv('../../Data/3/LeukocyteScore_Bindea_TMEcells.txt',
                sep='\t')

In [30]:
dav_data = pd.concat([luk_ovdata,dav_data], axis='columns', join='inner')

dav_data.head()

,LeukocyteMeth_score,CD4_mature,CD8_effector,NK_cells,B_cells,T_regs,Dendritics,CD8_effector_NK_cells,Macrophages,Macrophages_M2,Macrophages_M1
TCGA.04.1348.01,0.150949,0.022657,-0.133686,-0.139781,-0.045055,-0.014131,0.132643,-0.010075,0.413101,0.262198,0.369973
TCGA.04.1357.01,0.267944,0.103612,0.051358,0.004287,0.148263,-0.014680,0.148268,0.129134,0.450268,0.304542,0.433461
TCGA.04.1362.01,0.084071,-0.124390,-0.369165,-0.346159,-0.209140,-0.095841,-0.049672,-0.217201,0.333928,0.169423,0.170850
TCGA.04.1364.01,0.033465,-0.182340,-0.436440,-0.467786,-0.127647,-0.222235,-0.097677,-0.358318,-0.022455,-0.080364,-0.112248
TCGA.04.1365.01,0.141471,-0.042293,-0.155227,-0.146163,-0.005805,-0.108218,0.122148,-0.045398,0.390748,0.213862,0.348351


In [31]:
dav_data.to_csv('../../Data/3/LeukocyteScore_Davoli_TMEcells.txt',
                sep='\t')

In [32]:
dan_data = pd.concat([luk_ovdata,dan_data], axis='columns', join='inner')

dan_data.head()

,LeukocyteMeth_score,B.cells,CD45,CD8.T.cells,Cytotoxic.cells,DC,Exhausted.CD8,Macrophages,Mast.cells,Neutrophils,NK.CD56dim.cells,NK.cells,T.cells,Th1.cells,Treg
TCGA.04.1348.01,0.150949,-0.213772,0.237833,0.038574,-0.004140,-0.136734,-0.062459,0.210535,-0.307145,-0.188507,-0.236342,-0.204308,-0.043640,-0.183136,-0.027349
TCGA.04.1357.01,0.267944,-0.089404,0.422715,0.236438,0.072817,-0.147556,0.032395,0.231140,-0.317968,-0.160218,-0.219044,-0.166882,0.097841,-0.108760,-0.031581
TCGA.04.1362.01,0.084071,-0.343642,0.006612,-0.226110,-0.209050,-0.232155,-0.209200,0.050957,-0.335683,-0.212186,-0.363609,-0.312521,-0.232933,-0.319826,-0.099027
TCGA.04.1364.01,0.033465,-0.189344,-0.248201,-0.239439,-0.334968,-0.263151,-0.310654,-0.187723,-0.386647,-0.322938,-0.450434,-0.291942,-0.313260,-0.528989,-0.216356
TCGA.04.1365.01,0.141471,-0.231293,0.175730,-0.021701,-0.037648,-0.180634,-0.037813,0.163545,-0.339051,-0.145892,-0.258236,-0.259060,-0.049737,-0.199905,-0.078449


In [33]:
dan_data.to_csv('../../Data/3/LeukocyteScore_Danaher_TMEcells.txt',
                sep='\t')

In [34]:
con_data = pd.concat([luk_ovdata,con_data], axis='columns', join='inner')

con_data.head()

,LeukocyteMeth_score,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
TCGA.04.1348.01,0.150949,0.116094,0.015198,0.177872,-0.021979,0.163273,-0.026367,0.308360,0.265947,0.263728,0.227799,0.156372,0.040087,-0.042706,0.328188,0.081508,0.097530,0.064612,0.111321
TCGA.04.1357.01,0.267944,0.222905,0.142556,0.264784,0.018350,0.254335,0.120839,0.335431,0.325181,0.353313,0.315416,0.237937,0.200891,0.022803,0.392530,0.235109,0.247585,0.215137,0.264070
TCGA.04.1362.01,0.084071,-0.053375,-0.313831,-0.012503,-0.103150,-0.012369,-0.080783,0.140036,0.075724,0.076199,-0.024923,0.037650,-0.198247,-0.157238,0.236237,-0.127952,-0.126697,-0.173251,-0.171235
TCGA.04.1364.01,0.033465,-0.242056,-0.479257,-0.241447,-0.133003,-0.177319,-0.101909,-0.041094,-0.120676,-0.091007,-0.125996,-0.196706,-0.399216,-0.361281,0.117505,-0.298982,-0.283349,-0.286931,-0.398547
TCGA.04.1365.01,0.141471,0.043992,-0.059181,0.126954,-0.035426,0.137609,-0.023786,0.265740,0.244971,0.217097,0.192818,0.127383,-0.014752,-0.029825,0.333373,0.027088,0.039795,0.031051,0.039815


In [35]:
con_data.to_csv('../../Data/3/LeukocyteScore_ConsensusTME_TMEcells.txt',
                sep='\t')

In [36]:
cib_data = pd.concat([luk_ovdata,cib_data], axis='columns', join='inner')

cib_data.head()

,LeukocyteMeth_score,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells follicular helper,T cells regulatory (Tregs),...,Monocytes,Macrophages M0,Macrophages M1,Macrophages M2,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils
TCGA.04.1348.01,0.150949,0.000795,0.000830,0.000000,0.025868,0.0,0.038003,0.0,0.043519,0.026832,...,0.005156,0.104983,0.024237,0.062198,0.000000,0.000000,0.000000,0.014234,0.0,0.000000
TCGA.04.1357.01,0.267944,0.015474,0.000000,0.000000,0.083101,0.0,0.120940,0.0,0.053592,0.052900,...,0.006448,0.057623,0.062343,0.124707,0.001035,0.000000,0.014600,0.000000,0.0,0.000000
TCGA.04.1362.01,0.084071,0.008684,0.000000,0.000058,0.000000,0.0,0.049975,0.0,0.010518,0.000000,...,0.016486,0.000000,0.004502,0.058745,0.000000,0.017518,0.000000,0.001998,0.0,0.000000
TCGA.04.1364.01,0.033465,0.018040,0.000000,0.001023,0.000000,0.0,0.004297,0.0,0.001651,0.002840,...,0.000000,0.017076,0.000000,0.005596,0.000000,0.000000,0.000046,0.000000,0.0,0.000000
TCGA.04.1365.01,0.141471,0.001747,0.000229,0.000000,0.000000,0.0,0.072325,0.0,0.048944,0.020026,...,0.003811,0.080549,0.054699,0.034943,0.000000,0.000000,0.002676,0.000000,0.0,0.000605


In [37]:
cib_data.to_csv('../../Data/3/LeukocyteScore_cibersort_TMEcells.txt',
                sep='\t')

In [38]:
mcp_data = pd.concat([luk_ovdata,mcp_data], axis='columns', join='inner')

mcp_data.head()

,LeukocyteMeth_score,T cells,CD8 T cells,Cytotoxic lymphocytes,NK cells,B lineage,Monocytic lineage,Myeloid dendritic cells,Neutrophils,Endothelial cells,Fibroblasts
TCGA.04.1348.01,0.150949,140.379514,82.0113,122.446500,5.310562,61.407637,552.419614,34.31030,185.031820,227.841865,5541.236563
TCGA.04.1357.01,0.267944,284.691221,326.8229,289.747943,10.823550,220.784525,897.167057,154.29686,245.269087,204.511719,9558.854162
TCGA.04.1362.01,0.084071,217.392593,5.2953,20.046643,5.088475,38.763562,313.796143,12.97356,193.257807,257.996087,3400.404175
TCGA.04.1364.01,0.033465,41.826871,19.9291,1.877800,4.187238,98.479450,114.849786,5.85152,158.810920,236.205152,2706.700838
TCGA.04.1365.01,0.141471,167.179664,66.2950,107.277386,7.533512,92.021987,462.092157,40.43996,185.947460,201.272332,3607.766325


In [39]:
mcp_data.to_csv('../../Data/3/LeukocyteScore_mcp_TMEcells.txt',
                sep='\t')

In [40]:
tim_data = pd.concat([luk_ovdata,tim_data], axis='columns', join='inner')

tim_data.head()

,LeukocyteMeth_score,B_cell,T_cell.CD4,T_cell.CD8,Neutrophil,Macrophage,DC
sampleID,,,,,,,
TCGA.04.1348.01,0.150949,0.284869,0.206343,0.138018,0.187014,0.000000,0.575930
TCGA.04.1357.01,0.267944,0.605259,0.259462,0.024222,0.100005,0.000000,0.806742
TCGA.04.1362.01,0.084071,0.159977,0.144421,0.263358,0.100429,0.107244,0.338729
TCGA.04.1364.01,0.033465,0.149258,0.215276,0.022342,0.000000,0.041678,0.169301
TCGA.04.1365.01,0.141471,0.041258,0.086683,0.282746,0.195290,0.000000,0.533185


In [41]:
tim_data.to_csv('../../Data/3/LeukocyteScore_timer_TMEcells.txt',
                sep='\t')

In [42]:
xce_data = pd.concat([luk_ovdata,xce_data], axis='columns', join='inner')

xce_data.head()

,LeukocyteMeth_score,Adipocytes,Astrocytes,B-cells,Basophils,CD4+ T-cells,CD4+ Tcm,CD4+ Tem,CD4+ memory T-cells,CD4+ naive T-cells,...,Th2 cells,Tregs,aDC,cDC,iDC,ly Endothelial cells,mv Endothelial cells,naive B-cells,pDC,pro B-cells
TCGA.04.1348.01,0.150949,0.0000,0.0494,0.0535,0.1007,0.0,0.0000,0.0000,0.0074,0.0000,...,0.1211,0.0000,0.3265,0.0287,0.0,0.0000,0.0098,0.0,0.0199,0.0442
TCGA.04.1357.01,0.267944,0.0000,0.0301,0.1330,0.0486,0.0,0.0034,0.0458,0.0087,0.0227,...,0.0081,0.0057,0.3852,0.1140,0.0,0.0000,0.0037,0.0,0.0382,0.0015
TCGA.04.1362.01,0.084071,0.0000,0.0506,0.0039,0.0393,0.0,0.0239,0.0000,0.0000,0.0000,...,0.0780,0.0000,0.1563,0.0125,0.0,0.0019,0.0032,0.0,0.0070,0.0116
TCGA.04.1364.01,0.033465,0.0016,0.0654,0.0000,0.0452,0.0,0.0038,0.0000,0.0000,0.0000,...,0.1076,0.0000,0.0000,0.0000,0.0,0.0021,0.0117,0.0,0.0000,0.0390
TCGA.04.1365.01,0.141471,0.0000,0.0612,0.0179,0.0906,0.0,0.0000,0.0000,0.0121,0.0000,...,0.1709,0.0000,0.2709,0.0289,0.0,0.0000,0.0039,0.0,0.0155,0.0276


In [43]:
xce_data.to_csv('../../Data/3/LeukocyteScore_xcell_TMEcells.txt',
                sep='\t')

# End script